In [1]:
import librosa
import librosa.display
import scipy as sp
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
# from google.colab import drive
# drive.mount("/content/drive")
# %cd "/content/drive/MyDrive/Uchile_Sonido/ML/04 - Modelos clásicos de machine learning: teoría e implementación"

Recuperando porciones de audio borradas!

[Tutorial](https://mashimo.wordpress.com/2018/02/17/recover-audio-using-linear-regression/)

In [2]:
!ls

Actividad_Guiada.ipynb
OriginalTestClip.wav
clase_2_regresion_lineal_cuadratica.ipynb
free-spoken-digit-dataset-master


Descarga el dataset ejecutando la siguiente celda

In [3]:
#ad the dataset
#!wget https://github.com/Jakobovski/free-spoken-digit-dataset/archive/refs/heads/master.zip

#unzip the dataset folder
#! unzip master.zip

Cargando los audios

In [4]:
import scipy.io.wavfile as wavfile
import os

In [5]:
!ls

Actividad_Guiada.ipynb
OriginalTestClip.wav
clase_2_regresion_lineal_cuadratica.ipynb
free-spoken-digit-dataset-master


In [6]:
import os

In [7]:
os.getcwd()

'/Users/rodolfo.lobo/Desktop/CURSO_ML/Labs/clase_02'

In [8]:
zero = []
directory = "/Users/rodolfo.lobo/Desktop/CURSO_ML/Labs/clase_02/free-spoken-digit-dataset-master/recordings"
for fname in os.listdir(directory):
    if fname.startswith("0_jackson"):
        fullname = os.path.join(directory, fname)
        sample_rate, data = wavfile.read(fullname)
        zero.append( data )

In [9]:
len(zero)

50

In [10]:
zero[0]

array([343, 404, 454, ..., 420, 377, 345], dtype=int16)

In [11]:
sample_rate

8000

Preparando la data

In [12]:
import numpy as np
import pandas as pd

zeroDF = pd.DataFrame(zero, dtype=np.int16)
zeroDF.info()

/var/folders/p2/bdt9slt961vdyy05xknbjcs40000gr/T/ipykernel_79150/2301303207.py:4: FutureWarning: In a future version, passing float-dtype values containing NaN and an integer dtype will raise IntCastingNaNError (subclass of ValueError) instead of silently ignoring the passed dtype. To retain the old behavior, call Series(arr) or DataFrame(arr) without passing a dtype.
  zeroDF = pd.DataFrame(zero, dtype=np.int16)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Columns: 6273 entries, 0 to 6272
dtypes: float64(2186), int16(4087)
memory usage: 1.2 MB


In [13]:
zeroDF.head(2)

,0,1,2,3,4,5,6,7,8,9,...,6263,6264,6265,6266,6267,6268,6269,6270,6271,6272
0,343,404,454,477,477,462,425,393,364,311,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-311,-91,-140,-182,-271,-68,-235,-359,-129,-198,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Eliminando audios vacios

In [14]:
if zeroDF.isnull().values.any() == True:
  print("Preprocessing data: dropping all NaN")
  zeroDF.dropna(axis=1, inplace=True)
else:
  print("Preprocessing data: No NaN found!")

zero = zeroDF.values # this is an array

Preprocessing data: dropping all NaN


Retiramos solo un audio, aleatoriamente, para el resto usarlo como entrenamiento

In [15]:
from sklearn.utils.validation import check_random_state

rng   = check_random_state(7)
random_idx = rng.randint(zero.shape[0]) #random_idx = 47

test  = zero[random_idx] # the test sample
train = np.delete(zero, [random_idx], axis=0)

print(train.shape)
print(test.shape)

(49, 4087)
(4087,)


In [16]:
print(random_idx)

47


Guardamos el original

In [17]:
wavfile.write('OriginalTestClip.wav', sample_rate, test) # tan solo 1 audio para hacer test


In [18]:
len(test)

4087

In [19]:
test

array([-312, -335, -338, ...,  644,  815,  580], dtype=int16)

Les borramos la información del comienzo

In [20]:
Provided_Portion = 0.5 # fraccion de las muestras que borraremos
n_audio_samples = 4000 # 2 segundos borrados
test_samples = int(Provided_Portion * n_audio_samples)
X_test = test[0:test_samples] # first ones

In [21]:
X_test.shape

(2000,)

Escuchemos

In [22]:
import IPython . display as ipd

In [23]:
ipd.Audio(X_test, rate=8000)

Borremos un cuarto de la información

In [24]:
Provided_Portion = 0.25 # let's delete three quarters of the audio!

test_samples = int(Provided_Portion * n_audio_samples)
X_test = test[0:test_samples] # first ones

In [25]:
ipd.Audio(X_test, rate=8000)

Separado en datos de entrenamiento y test. Recordando que en test solo tendremos un solo audio.

In [26]:
y_test = test[test_samples:] # remaining audio part is the label

X_train = train[:, 0:test_samples] # first ones: data
y_train = train[:, test_samples:]  # remaining ones: label

X_test = X_test.reshape(1,-1)
y_test = y_test.reshape(1,-1)

Escuchemos ejemplos del entrenamiento

Observe que todos los ejemplos son el mismo número, que ha sido leído por una persona en diferentes ocasiones.

In [27]:
import random

In [28]:
random_choice = random.randint(0, 48)
ipd.Audio(train[random_choice,:], rate=8000)

In [29]:
len(train[random_choice,:])

4087

La primera porción del audio es el ejemplo y la segunda porción la clase o salida

In [30]:
X_train.shape

(49, 1000)

In [31]:
y_train.shape

(49, 3087)

Creamos un modelo de regresión lineal

Recordemos que nuestro problema tiene la forma

\begin{equation}
\text{min} || \theta X + b = Y ||_{2}^{2}
\end{equation}

donde $\theta\in \mathbb{R}^{3087\times 1000}$, $X\in \mathbb{R}^{1000\times 49}$, $b \in \mathbb{R}^{3087 \times 49}$ repetido 49 veces para resolver esta parte del problema e $Y\in \mathbb{R}^{3087 \times 49}$ en el momento del entrenamiento. Una vez resuelto el problema de minimización, encontramos la matriz $\theta$. Finalmente, dado un ejemplo de audio cortado $\mathbf{c}\in \mathbb{R}^{1000 \times 1}$ queremos completarlo para obtener el audio real o ideal dado por $\mathbf{r}\in \mathbb{R}^{3087 \times 1}$ y $b \in \mathbb{R}^{3087\times 1}$, a través de:

\begin{equation}
\theta \mathbf{c}+ b = \mathbf{r}
\end{equation}



In [32]:
from sklearn import linear_model

model = linear_model.LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [33]:
model.get_params()

{'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'positive': False}

In [46]:
model.fit(X_train, y_train)
print(model.intercept_, model.coef_, model.score(X_train, y_train))

[  95.54757888  140.75024121  -38.34045389 ...  158.78373094 -178.81695808
 -571.78206978] [[-4.96414007e-03 -5.35098816e-03 -5.18471595e-03 ...  1.32791956e-01
   1.35588326e-01  1.71018363e-01]
 [-7.40837878e-04 -6.45429450e-04 -1.26106093e-04 ...  9.66387714e-02
   1.24307486e-01  1.59588719e-01]
 [-8.48752376e-03 -1.08928275e-02 -1.13215590e-02 ...  6.69210242e-02
   1.07196824e-01  1.61340126e-01]
 ...
 [ 9.07254129e-03  1.10229517e-02  1.09652373e-02 ... -6.81860788e-03
  -1.94730765e-03  1.31026815e-02]
 [-3.03022438e-04  3.72729952e-04 -8.88737007e-04 ... -7.36657012e-02
  -3.21707196e-02 -2.51104525e-03]
 [-1.16553930e-02 -1.28443926e-02 -1.50356423e-02 ... -1.25589960e-01
  -5.69328630e-02 -1.26491669e-02]] 1.0


Dimensión de la matriz de coeficientes $\theta$

In [35]:
model.coef_.shape

(3087, 1000)

El modelo obtuvo un score de 1.0 llamado coeficiente de determinación o $R^2$

In [36]:
model.score(X_train, y_train)

1.0

In [37]:
#help(linear_model.LinearRegression())

Valor de la intersección

In [38]:
model.intercept_.shape

(3087,)

Testeamos, escuchemos nuevamente el audio reducido

In [39]:
len(X_test[0])

1000

In [40]:
ipd.Audio(X_test[0], rate=8000)

Entramos con el audio reducido y la salida será un audio del tamaño

In [41]:
y_test_prediction = model.predict(X_test)

In [42]:
len(y_test_prediction[0])

3087

In [43]:
y_test_prediction = y_test_prediction.astype(dtype=np.int16)

Escuchemos el resultado

In [44]:
completed_clip = np.hstack((X_test, y_test_prediction))

In [45]:
ipd.Audio(completed_clip, rate=8000)